In [138]:
# The code was removed by Watson Studio for sharing.

[Row(mpg='18', cylinders='8', displacement='307', horsepower='130', weight='3504', acceleration='12', model_year='70'),
 Row(mpg='15', cylinders='8', displacement='350', horsepower='165', weight='3693', acceleration='11.5', model_year='70'),
 Row(mpg='18', cylinders='8', displacement='318', horsepower='150', weight='3436', acceleration='11', model_year='70'),
 Row(mpg='16', cylinders='8', displacement='304', horsepower='150', weight='3433', acceleration='12', model_year='70'),
 Row(mpg='17', cylinders='8', displacement='302', horsepower='140', weight='3449', acceleration='10.5', model_year='70')]

In [72]:
#convert to integer & double

from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType

autompg_df=autompg_df.withColumn("mpg", autompg_df["mpg"].cast(IntegerType()))
autompg_df=autompg_df.withColumn("cylinders", autompg_df["cylinders"].cast(IntegerType()))
autompg_df=autompg_df.withColumn("displacement", autompg_df["displacement"].cast(IntegerType()))
autompg_df=autompg_df.withColumn("horsepower", autompg_df["horsepower"].cast(IntegerType()))
autompg_df=autompg_df.withColumn("weight", autompg_df["weight"].cast(IntegerType()))
autompg_df=autompg_df.withColumn("acceleration", autompg_df["acceleration"].cast(DoubleType()))
autompg_df=autompg_df.withColumn("model_year", autompg_df["model_year"].cast(IntegerType()))

In [73]:
#print schema to check data type

autompg_df.cache()
autompg_df.printSchema()

root
 |-- mpg: integer (nullable = true)
 |-- cylinders: integer (nullable = true)
 |-- displacement: integer (nullable = true)
 |-- horsepower: integer (nullable = true)
 |-- weight: integer (nullable = true)
 |-- acceleration: double (nullable = true)
 |-- model_year: integer (nullable = true)



In [74]:
#statistik deskriptif

autompg_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
mpg,392,23.283163265306122,7.745895518598281,9,46
cylinders,392,5.471938775510204,1.7057832474527845,3,8
displacement,392,194.41071428571428,104.64519123492178,68,455
horsepower,392,104.46938775510205,38.49115993282846,46,230
weight,392,2977.5841836734694,849.4025600429486,1613,5140
acceleration,392,15.541326530612228,2.75886411918808,8.0,24.8
model_year,392,75.9795918367347,3.683736543577868,70,82


In [75]:
#find data correlation

import six
for i in autompg_df.columns:
    if not( isinstance(autompg_df.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to mpg for ", i, autompg_df.stat.corr('mpg',i))

Correlation to mpg for  mpg 1.0
Correlation to mpg for  cylinders -0.7801431847418415
Correlation to mpg for  displacement -0.8067650504345276
Correlation to mpg for  horsepower -0.7794147299944597
Correlation to mpg for  weight -0.8350800096091144
Correlation to mpg for  acceleration 0.4237641938905716
Correlation to mpg for  model_year 0.5690076613980674


In [76]:
#mapping features and mpg

from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols = ['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year'], outputCol = 'features')
vhouse_df = vectorAssembler.transform(autompg_df)
vhouse_df = vhouse_df.select(['features', 'mpg'])
vhouse_df.show(3)

+--------------------+---+
|            features|mpg|
+--------------------+---+
|[8.0,307.0,130.0,...| 18|
|[8.0,350.0,165.0,...| 15|
|[8.0,318.0,150.0,...| 18|
+--------------------+---+
only showing top 3 rows



In [77]:
#split training to test

splits = vhouse_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [111]:
#linear regression
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol = 'features', labelCol='mpg', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [-0.0579200582013,-0.00369747009786,-0.00314446845492,-0.00579259422779,0.0,0.69412952953]
Intercept: -10.760170895222297


In [112]:
#root mean and r square

trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 3.453549
r2: 0.807264


In [113]:
#describe data training

train_df.describe().show()

+-------+-----------------+
|summary|              mpg|
+-------+-----------------+
|  count|              288|
|   mean|23.35763888888889|
| stddev|7.880236646025767|
|    min|                9|
|    max|               46|
+-------+-----------------+



In [114]:
#akurasi prediksi test data

lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","mpg","features").show(10)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="mpg",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+---+--------------------+
|        prediction|mpg|            features|
+------------------+---+--------------------+
|30.005083508953724| 23|[3.0,70.0,100.0,2...|
|26.116497266499614| 21|[3.0,80.0,110.0,2...|
| 28.45932418340942| 29|[4.0,68.0,49.0,18...|
|30.377226088449888| 31|[4.0,76.0,52.0,16...|
| 31.20003760583525| 32|[4.0,78.0,52.0,19...|
|28.500092063806214| 26|[4.0,79.0,67.0,19...|
|29.496266544701307| 29|[4.0,85.0,52.0,20...|
|31.797462005821835| 40|[4.0,85.0,65.0,21...|
|30.625184373599474| 39|[4.0,85.0,70.0,20...|
|25.765281417007465| 30|[4.0,88.0,76.0,20...|
+------------------+---+--------------------+
only showing top 10 rows

R Squared (R2) on test data = 0.789473


In [115]:
#find RMSE on the test data set

test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 3.37633


In [116]:
#Decision Tree

from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'mpg')
dt_model = dt.fit(train_df)
dt_predictions = dt_model.transform(test_df)
dt_evaluator = RegressionEvaluator(
    labelCol="mpg", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 3.39005


In [118]:
#mana yang paling berpengaruh

dt_model.featureImportances

SparseVector(6, {1: 0.6602, 2: 0.1826, 3: 0.0541, 4: 0.0026, 5: 0.1004})

In [119]:
autompg_df.take(1)

[Row(mpg=18, cylinders=8, displacement=307, horsepower=130, weight=3504, acceleration=12.0, model_year=70)]

In [120]:
#gradient-boosted tree regression

from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'mpg', maxIter=10)
gbt_model = gbt.fit(train_df)
gbt_predictions = gbt_model.transform(test_df)
gbt_predictions.select('prediction', 'mpg', 'features').show(5)

+------------------+---+--------------------+
|        prediction|mpg|            features|
+------------------+---+--------------------+
| 32.50757655055839| 23|[3.0,70.0,100.0,2...|
|18.946737087148758| 21|[3.0,80.0,110.0,2...|
|30.764691275277873| 29|[4.0,68.0,49.0,18...|
| 34.30907384961368| 31|[4.0,76.0,52.0,16...|
|36.897961630737356| 32|[4.0,78.0,52.0,19...|
+------------------+---+--------------------+
only showing top 5 rows



In [104]:
#Evaluate GBT

gbt_evaluator = RegressionEvaluator(
    labelCol="mpg", predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 3.23249


In [134]:
#logistic regression
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

nb = NaiveBayes(featuresCol = 'features', labelCol='mpg', smoothing=1.0, modelType="multinomial")
model = nb.fit(train_df)


In [135]:
# select example rows to display.
predictions = model.transform(test_df)
predictions.show()

+--------------------+---+--------------------+--------------------+----------+
|            features|mpg|       rawPrediction|         probability|prediction|
+--------------------+---+--------------------+--------------------+----------+
|[3.0,70.0,100.0,2...| 23|[-1259.6566725228...|[1.03651415457028...|      22.0|
|[3.0,80.0,110.0,2...| 21|[-1349.7795926595...|[3.08234809356765...|      15.0|
|[4.0,68.0,49.0,18...| 29|[-1034.5269489371...|[1.01285903389234...|      32.0|
|[4.0,76.0,52.0,16...| 31|[-1028.8839394635...|[1.76150459977023...|      30.0|
|[4.0,78.0,52.0,19...| 32|[-1108.1614003062...|[8.99310228649071...|      33.0|
|[4.0,79.0,67.0,19...| 26|[-1118.9481798674...|[2.91582943032005...|      22.0|
|[4.0,85.0,52.0,20...| 29|[-1141.2159114451...|[6.72989170062788...|      32.0|
|[4.0,85.0,65.0,21...| 40|[-1193.6642129306...|[5.54400766844986...|      22.0|
|[4.0,85.0,70.0,20...| 39|[-1193.4649265949...|[1.50810445215533...|      22.0|
|[4.0,88.0,76.0,20...| 30|[-1168.0980192

In [137]:
# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="mpg", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.009615384615384616
